In [0]:
import time
from pyspark.sql import functions as F

In [0]:

source_path = "/Volumes/raw/f1_world/streaming_source/"
landing_path = "/Volumes/raw/f1_world/landing_zone/"

df_laps_source = spark.read.format("parquet").load(f"{source_path}lap_times")
df_pit_source = spark.read.format("parquet").load(f"{source_path}pit_stops")

laps_list = [row.lap for row in df_laps_source.select(F.col("lap").cast("int")).distinct().sort("lap").collect()]

for current_lap in laps_list:
    start_time = time.time()

    batch_lap = df_laps_source.filter(F.col("lap") == current_lap)

    batch_lap.write.format("parquet").mode("append").save(f"{landing_path}lap_times")


    batch_pit = df_pit_source.filter(F.col("lap") == current_lap)

    if batch_pit.count()>0:
        batch_pit.write.format("parquet").mode("append").save(f"{landing_path}pit_stops")
        print(f"Volta {current_lap}: Dados de volta e Pit Stop enviados.")
    else:
        print(f"Volta {current_lap}: Dados de volta enviados.")

    time.sleep(3)

print("Fim da corrida")

final_files = ["constructor_results","constructor_standings","qualifying","results", "sprint_results"]

for file_name in final_files:
    try:
        df_final = spark.read.format("parquet").load(f"{source_path}{file_name}")

        df_final.write.format("parquet").mode("append").save(f"{landing_path}{file_name}")
        print(f"Arquivo {file_name} enviado.")
    except:
        print(f"Arquivo {file_name} não encontrado.")

